## CHAPTER6.다양한 데이터 타입 다루기
- 불리언 타입
- 수치 타입
- 문자열 타입
- date와 timestamp 타입
- null값 다루기
- 복합 데이터 타입
- 사용자 정의 함수

### API는 어디서 찾을까
데이터 변환용 함수
- DataFrame(Dataset) 메서드
> DataFrame은 Row 타입을 가진 Dataset -> Dataset의 메서드를 만나게 됨(내려받음?)\
> DataFrameStatFunctions: 다양한 통계적 함수 제공\
> DataFrameNaFunctions: NULL 데이터를 다루는 데 필요한 함수 제공

- Column 메서드
> alias, contains 같은 컬럼 관련된 여러가지 메서드 제공
> ColumnAPI는 스파크 문서 참고(https://spark.apache.org/docs/3.1.1/api/java/org/apache/spark/sql/Column.html)


In [10]:
df = spark.read.format("CSV")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("../../assets/presentations/week06/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable") # Session scope
spark.sql("SELECT * FROM dfTable LIMIT 10").show()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365| 

### 스파크 데이터 타입으로 변환하기
- lit 함수 사용하여 데이터 변환
- SQL에서는 스파크 데이터 타입으로 변환할 필요 없음 -> 값 직접 입력하여 사용

In [14]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

### 불리언 데이터 타입 다루기
- 필터링 작업의 기반으로 데이터 분석에 필수적
- and, or, true, false
- 일치 조건 이외 작다, 크다와 같은 비교 연산 조건 사용 가능

** 불리언 표현식을 만들 때 null 값 데이터 다루기(null값에 안전한 동치 테스트 수행 예제 제공)

    df.where(col("Description").eqNullSafe("hello")).show()


In [17]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365)\
    .select("InvoiceNo", "Description")\
    .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [23]:
df.where("InvoiceNo = 536365")\
    .show(5, False)
df.where("InvoiceNo <> 536365")\
    .show(5, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [24]:
# df.where(col("Description").eqNullSafe("hello")).show()

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+



### 수치형 데이터 타입 다루기
- pow: 표시된 지수만큼 컬럼 값 거듭제곱
- round/bround: 소수점 값이 정확히 중간값 이상이면 반올림 / 내림 함수
- DataFrame의 통계용 함수나 메서드를 사용가능(ex.피어슨 상관계수)
- 하나 이상의 컬럼에 대한 요약 통계 확인 가능(describe() 메서드)
    - 통계 스키마는 변경될 수 있으므로, describe 메서드는 콘솔 확인용으로만 사용할 것 권장
- StatFunrctions 패키지
    - stat 속성을 사용해 접근
    - ex1.qpproxQuantile: 데이터의 백분위수 계산
    - ex2.crosstab
    - ex3.monotonically_increasing_id: 모든 로우에 고유 ID 값 추가(0부터 시작하는 고윳값 생성)
- 함수는 새 버전 릴리스 마다 생겨남. API 문서 참고


In [26]:
# pow 함수 예제
from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col("Quantity")*col("UnitPrice"),2) + 5
df.select(expr("CustomerId"),fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [27]:
# round
from pyspark.sql.functions import lit, round, bround
df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [30]:
# 피어슨 상관계수 예제
from pyspark.sql.functions import corr

# df.stat.corr("Quantity","UnitPrice")
df.select(corr("Quantity","UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [32]:
# describe
df.describe().show()

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|               0.0|           12431.0|     Australia|
|    max|          C

In [36]:
# crosstab
# df.stat.crosstab("StockCode","Quantity").show(2)
# freqItems
df.stat.freqItems(["StockCode","Quantity"]).show()

+--------------------+--------------------+
| StockCode_freqItems|  Quantity_freqItems|
+--------------------+--------------------+
|[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+



In [41]:
# monotonically_increasing_id
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id(),"InvoiceNo","Description").show(2)

+-----------------------------+---------+--------------------+
|monotonically_increasing_id()|InvoiceNo|         Description|
+-----------------------------+---------+--------------------+
|                            0|   536365|WHITE HANGING HEA...|
|                            1|   536365| WHITE METAL LANTERN|
+-----------------------------+---------+--------------------+
only showing top 2 rows



### 문자열 데이터 타입 다루기
- 정규표현식 사용
    - java 정규 표현식 사용
    - regexp_extract: 정규 표현식에 해당하는 값 추출
    - regexp_replace: 정규 표현식에 해당하는 값 치환
- 데이터 추출
- 데이터 치환
- 문자열 존재 여부
- 대/소문자 변환 처리 등
- lower, upper, lpad, ltrim, rpad, rtrim ...
- translate: 교체 문자열에서 색인된 모든 문자열 치환/문자 단위 연산
- contains(스칼라) / instr(파이썬, SQL): 값 존재 여부 확인
- locate(파이썬): 문자열의 위치를 정술 반환(1부터 시작)


In [44]:
# 정규표현식 예제
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
    regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
    col("Description")).show(2)


+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [54]:
# locate 예제
from pyspark.sql.functions import expr,locate

simpleColors = ["black","white","red","green","blue"]
def color_locator(column, color_string):
    return locate(color_string.upper(), column)\
        .cast("boolean")\
        .alias("is_" + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
print(selectedColumns)
selectedColumns.append(expr("*"))

df.select(*selectedColumns).where(expr("is_white OR is_red"))\
    .select("Description").show(3, False)

[Column<'CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS is_black'>, Column<'CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS is_white'>, Column<'CAST(locate(RED, Description, 1) AS BOOLEAN) AS is_red'>, Column<'CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS is_green'>, Column<'CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS is_blue'>]
+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



### 날짜와 타임스탬프 데이터 타입 다루기
- date: 날짜
- timestamp: 날짜와 시간 정보 모두
- 날짜나 시간을 문자열로 저장 -> 런타임에 날짜 타입으로 변환하는 경우 있음
    - 시간대 설정이 필요하다면 spark.conf.sessionLocalTimeZone 속성을 로컬 시간대로 지정해 적용 가능
    - 시간대 포맷은 반드시 자바 TimeZone 포맷을 따라야 함.
- 날짜와 시간을 최대한 올바른 형태로 읽기 위해 노력(...)
- 특이한 포맷의 날짜와 시간데이터를 어쩔 수 없이 다뤄야 한다면, 단계별로 어떤 데이터 타입과 포맷을 유지하는지 정확히 파악 후 트랜스포메이션 적용해야 한다.
- TimestampType 클래스는 초 단위 정밀도까지만 지원
    - 밀리세컨드, 마이크로세컨드 단위를 다룬다면 Long 데이터 타입으로 변화냏 처리하는 우회정책 사용 필요
- date_sub, date_add: 날짜 빼기,더하기
- datetdiff: 두 날짜 사이의 일수 반환
- months_between: 두 날짜 사이의 개월 수 반환
- to_date: 문자열을 날짜로 변환, 날짜 포맷 지정 가능(날짜 포맷은 자바의 SimpleDateFormat 클래스가 지원하는 포맷 사용)
- to_timestamp: 반드시 날짜 포맷 지정해야 함.(yyyy-MM-dd HH:mm:ss 포맷 기본값 사용)
- 스파크가 날짜를 파싱할 수 없는 경우
    - 에러가 아닌 null 값을 반환
    - 데이터 포맷이 지정된 데이터에서 다른 포맷 데이터가 발생 가능
    - 정확한 포맷 지정 권장.
- 명시적 형변환 추천

In [60]:
# null값 반환
from pyspark.sql.functions import current_date, current_timestamp
from pyspark.sql.functions import to_date, lit
dateDF = spark.range(10)\
    .withColumn("today",current_date())\
    .withColumn("now", current_timestamp())
dateDF.select(to_date(lit("2016-20-12")), to_date(lit("2017-12-11"))).show(1) 
#년-일-월 포맷. 스파크가 의도에 맞게 인식 불가 + 파싱 불가로 null 반환

+-------------------+-------------------+
|to_date(2016-20-12)|to_date(2017-12-11)|
+-------------------+-------------------+
|               null|         2017-12-11|
+-------------------+-------------------+
only showing top 1 row



In [76]:
# 포맷 지정 # date3는 포맷 지정X로 잘못된 값 들어가는 케이스(월일 뒤바뀌었음)
from pyspark.sql.functions import to_timestamp
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_timestamp(lit("2017-20-12"), dateFormat).alias("date2"),
    to_timestamp(lit("2017-11-12")).alias("date3"))
cleanDateDF.createOrReplaceTempView("dateTable2")

spark.sql("SELECT date, date2, date3 FROM dateTable2").show()
# spark.sql("SELECT to_date(date,'yyyy-dd-MM'),to_date(date2,'yyyy-dd-MM'),to_date(date) FROM dateTable2").show()


+----------+-------------------+-------------------+
|      date|              date2|              date3|
+----------+-------------------+-------------------+
|2017-11-12|2017-12-20 00:00:00|2017-11-12 00:00:00|
+----------+-------------------+-------------------+



### null값 다루기
- 스파크에서는 빈 문자열이나 대체 값 대신 null값을 사용해야 최적화를 수행할 수 있음
- null 값을 허용하지 않는다 선언해도 강제성 없음.
- nullable 속성은 스파크 SQL 옵티마이저가 해당 컬럼을 제어하는 동작을 단순하게 도울 뿐
- coalesce: 인수로 지정한 여러 컬럼 중 null이 아닌 첫번 째 값을 반환. 모든 컬럼이 null이 아니라면 첫 번재 컬럼 반환
- ifnull: 첫 번째 값이 null이면 두 번째 값 반환. 첫 번째 값이 null이 아니라면 첫 번째 값 반환
- nullif: 두 값이 같으면 null 반환. 두 값이 다르면 첫 번째 값을 반환
- nvl: 첫 번째 값이 null이면 두 번째 값을 반환. 첫 번째 값이 null이 아니라면 첫 번째 값 반환
- nvl2: 첫 번째 값이 null이 아니면 두 번째 값을 반환. 첫 번째 값이 null이면 세 번째 인수로 지정된 값을 반환

### drop
- null값을 가진 로우 제거하는 가장 간단한 함수. null값을 가진 모든 로우 제거
- drop 메서드의 인수로 any로 지정한 경우 로우의 컬럼 값 중 하나라도 null값을 가지면 해당 로우를 제거. 
- all을 지정한 경우 모든 컬럼의 값이 null 혹은 NaN인 경우에만 해당 로우를 제거
- drop 메서드에 배열 형태의 컬럼을 인수로 전달해 적용 가능

### fill
- 하나 이상의 컬럼을 특정 값으로 채울 수 있음.
- 채워 넣을 값과 컬럼 집합으로 구성된 맵을 인수로 사용
- 다수의 컬럼에 적용하고 싶다면 적용하고자 하는 컬럼명을 배열로 만들어 인수로 사용

### replace
- 조건에 따라 다른 값으로 대체
- replace 메서드를 사용하는 경우 변경하고자 하는 값과 원래 값의 데이터 타입이 같아야 함.

In [84]:
# drop
df.na.drop()
df.na.drop("any")
df.na.drop("all")
df.na.drop("all",subset=["StockCode","InvoiceNo"])

# fill
df.na.fill("All Null values become this string")

# replace
df.na.replace([""], ["UNKNOWN"], "Description")

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string]

### 정렬하기
- asc_nulls_first, desc_nulls_first, asc_nulls_last, desc_nulls_last
- DataFrame을 정렬할 때 null 값 표시 기준 지정 가능

### 복합 데이터 타입 다루기
- 구조체(struct)
    - DataFrame 내부의 DataFrame
    - 쿼리문에서 다수의 컬럼을 괄호로 묶어 구조체 생성 가능
    - 차이점: 문법에 . 혹은 getField 메서드 사용
- 배열(array)
    - split: 구분자를 인수로 전달하여 배열로 변환
    - size: 배열의 길이 조회
    - array_contains: 배열에 특정 값이 존재하는지 확인
    - explode: 배열 타입의 컬럼을 입력 받아, 입력된 컬럼의 배열값에 포함된 모든 값을 로우로 변환. 나머지 컬럼값은 중복되어 표시
- 맵(map)
    - 함수와 컬럼의 키-값 쌍을 이용해 생성
    - 배열과 동일한 방법으로 값을 선택할 수 있음.
    - 적합한 키를 사용해 데이터 조회
    - 해당 키가 없다면 null 값을 반환
    - map 타입은 분해하여 컬럼으로 변환 가능

In [90]:
# struct
from pyspark.sql.functions import struct

complexDF = df.select(struct("Description","InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")
complexDF.select("complex.Description")
complexDF.select(col("complex").getField("Description")).show(1)
complexDF.select("complex.*").show(1)

+--------------------+
| complex.Description|
+--------------------+
|WHITE HANGING HEA...|
+--------------------+
only showing top 1 row

+--------------------+---------+
|         Description|InvoiceNo|
+--------------------+---------+
|WHITE HANGING HEA...|   536365|
+--------------------+---------+
only showing top 1 row



In [98]:
# array
from pyspark.sql.functions import split
df.select(split(col("Description"), " ").alias("array_col"))\
    .selectExpr("array_col[0]").show(2)

from pyspark.sql.functions import size
df.select(size(split(col("Description"), " "))).show(2)

from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)

from pyspark.sql.functions import explode
df.withColumn("splitted", split(col("Description"), " "))\
    .withColumn("exploded", explode(col("splitted")))\
    .select("Description","InvoiceNo","exploded").show(10)

+------------+
|array_col[0]|
+------------+
|       WHITE|
|       WHITE|
+------------+
only showing top 2 rows

+-------------------------------+
|size(split(Description,  , -1))|
+-------------------------------+
|                              5|
|                              3|
+-------------------------------+
only showing top 2 rows

+------------------------------------------------+
|array_contains(split(Description,  , -1), WHITE)|
+------------------------------------------------+
|                                            true|
|                                            true|
+------------------------------------------------+
only showing top 2 rows

+--------------------+---------+--------+
|         Description|InvoiceNo|exploded|
+--------------------+---------+--------+
|WHITE HANGING HEA...|   536365|   WHITE|
|WHITE HANGING HEA...|   536365| HANGING|
|WHITE HANGING HEA...|   536365|   HEART|
|WHITE HANGING HEA...|   536365| T-LIGHT|
|WHITE HANGING HEA...|   536365

In [104]:
# map
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
    .show(2)

df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("complex_map['WHITE METAL LANTERN']").show(2)

df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
    .selectExpr("explode(complex_map)").show(2)

+--------------------+
|         complex_map|
+--------------------+
|{WHITE HANGING HE...|
|{WHITE METAL LANT...|
+--------------------+
only showing top 2 rows

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                          536365|
+--------------------------------+
only showing top 2 rows

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



### JSON 다루기
- 스파크에서 문자열 형태의 JSON 직접 조작 가능
- JSON 파싱
- JSON 객체로 만들 수 있음.
- get_json_object: JSON 객체(딕셔너리나 배열)를 인라인 쿼리로 조회 가능
- json_tuple
- to_json: StructType을 JSON 문자열로 변경 가능
- from_json: JSON 문자열을 다시 객체로 변환. 파라미터로 반드시 스키마 지정 필요. 

### 사용자 정의 함수
- UserDefinedFunction, UDF
- 파이썬이나 스칼라 그리고 외부 라이브러리를 사용해 사용자가 원하는 형태로 트랜스포메이션을 만들 수 있게끔 한다.
- 여러가지 프로그래밍 언어로 개발할 수 있어 강력
    - 스칼라, 파이썬 자바로 UDF 개발 가능. 하지만 언어별로 성능에 영향을 미칠 수 있으므로 주의 필요
- 레코드별로 데이터를 처리하는 함수이기에 독특한 포맷이나 도메인에 특화된 언어를 사용하지 않음.
- 특정 SparkWession이나 Context에서 사용할 수 있도록 임시 함수 형태로 등록
<img src="../../assets/presentations/week06/python_udf.png" width="500px" height="250px" title="catalyst_optimizer"/>

### Hive UDF
- 하이브 문법을 사용해서 만든 UDF/UDAF 사용 가능
- SparkSession.builder().enableHiveSupport(): 하이브 지원 기능 활성화(명시 필수)